In [1]:
from selenium import webdriver as wd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
# 명시적 대기를 위해 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time


In [2]:
# 디비 처리, 연결, 해제, 검색어 가져오기, 데이터 삽입
import pymysql as my

class DBHelper:
    '''
    맴버변수 : 커넥션 
    '''
    conn = None
    '''
    생성자
    '''
    def __init__(self):
        self.db_init()
    '''
    맴버 함수
    '''
    def db_init(self):
        self.conn = my.connect(
                        host='localhost',
                        user='practice',
                        password='6725',
                        db='pythondb',
                        charset='utf8',
                        cursorclass=my.cursors.DictCursor )
    
    def db_free(self):
        if self.conn:
            self.conn.close()

    # 검색 키워드 가져오기 => 웹에서 검색
    def db_selectKeyword(self):
        # 커서 오픈
        # with => 닫기를 처리를 자동으로 처리해준다 => I/O 많이 사용
        rows = None
        with self.conn.cursor() as cursor:
            sql  = "select * from tbl_keyword;"
            cursor.execute(sql)
            rows = cursor.fetchall()
            print(rows)
        return rows
        
    def db_insertCrawlingData(self, title, price, area, contents, keyword ):
        with self.conn.cursor() as cursor:
            sql = "insert into tbl_crawlingdata (title, price, area, contents, keyword) values( %s,%s,%s,%s,%s );"
            cursor.execute(sql, (title, price, area, contents, keyword) )
        self.conn.commit()

In [3]:
# 상품 정보를 담는 클레스
class TourInfo:
    # 맴버변수 (실제 컬럼보다는 작게 세팅했음)
    title = ''
    price = ''
    area  = ''
    link  = ''
    img   = ''
    contents = ''
    # 생성자
    def __init__(self, title, price, area, link, img, contents=None ):
        self.title = title
        self.price = price
        self.area  = area
        self.link  = link
        self.img   = img
        self.contents = contents

In [4]:
# 사전에 필요한 정보를 로드 => 디비혹스 쉘, 베치 파일에서 인자로 받아서 세팅

main_url = 'http://tour.interpark.com/' 
keyword  = '로마'
# 상품 정보를 담는 리스트 (TourInfo 리스트)
tour_list = []

In [5]:
# 윈도우용
driver = wd.Chrome(executable_path='.\\webdriver\\chromedriver.exe')

<ipython-input-5-be4c1778cd5a>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome(executable_path='.\\webdriver\\chromedriver.exe')


In [6]:
# 사이트 접속 (get)
driver.get(main_url)
# 검색창을 찾아서 검색어 입력
# id : SearchGNBText
driver.find_element_by_id('SearchGNBText').send_keys(keyword)
# 수정할경우 => 뒤에 내용이 붙어버림 => .clear() -> send_keys('내용')
# 검색 버튼 클릭
driver.find_element_by_css_selector('button.search-btn').click()

C:\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:390: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:670: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [7]:
# 잠시 대기 => 페이가 로드되고 나서 즉각적으로 데이터를 획득 하는 행위는 
# 명시적 대기 => 특정 요소가 로케이트(발결된때까지) 대기
try:
    element = WebDriverWait(driver, 10).until(
        # 지정한 한개 요소가 올라면 웨이트 종료
        EC.presence_of_element_located( (By.CLASS_NAME, 'oTravelBox') )
    )
except Exception as e:
    print( '오류 발생', e)
# 암묵적 대기 => DOM이 다 로드 될때까지 대기 하고 먼저 로드되면 바로 진행
# 요소를 찾을 특정 시간 동안 DOM 풀링을 지시 예를 들어 10 초이내 라로 
# 발견 되면 진행
driver.implicitly_wait( 10 )
# 절대기 대기 => time.sleep(10) -> 클라우드 페어(디도스 방어  솔류션)
# 더보기 눌러서 => 게시판 진입 
driver.find_element_by_css_selector('.oTravelBox>.boxList>.moreBtnWrap>.moreBtn').click()

In [8]:
# searchModule.SetCategoryList(1, '') 스크립트 실행
# 16은 임시값, 게시물을 넘어갔을때 현상을 확인차
for page in range(1, 2): # 2):
    try:
        # 자바스크립트 구동하기
        driver.execute_script("searchModule.SetCategoryList(%s, '')" % page)
        time.sleep(2)
        print("%s 페이지 이동" % page)
        #############################################################
        # 여러 사이트에서 정보를 수집할 경우 공통 정보 정의 단계 필요
        # 상품명, 코멘트, 기간1, 기간2, 가격, 평점, 썸네일, 링크(상품상세정보)
        boxItems = driver.find_elements_by_css_selector('.oTravelBox>.boxList>li')
        # 상품 하나 하나 접근
        for li in boxItems:
            # 이미지를 링크값을 사용할것인가? 
            # 직접 다운로드 해서 우리 서버에 업로드(ftp) 할것인가?
            print( '썸네임', li.find_element_by_css_selector('img').get_attribute('src') )
            print( '링크', li.find_element_by_css_selector('a').get_attribute('onclick') )
            print( '상품명', li.find_element_by_css_selector('h5.proTit').text )
            print( '코멘트', li.find_element_by_css_selector('.proSub').text )
            print( '가격',   li.find_element_by_css_selector('.proPrice').text )
            area = ''
            for info in li.find_elements_by_css_selector('.info-row .proInfo'):
                print(  info.text )
            print('='*100)
            # 데이터 모음
            # li.find_elements_by_css_selector('.info-row .proInfo')[1].text
            # 데이터가 부족하거나 없을수도 있으므로 직접 인덱스로 표현은 위험성이 있음
            obj = TourInfo(  
                li.find_element_by_css_selector('h5.proTit').text,
                li.find_element_by_css_selector('.proPrice').text,
                li.find_elements_by_css_selector('.info-row .proInfo')[1].text,
                li.find_element_by_css_selector('a').get_attribute('onclick'),
                li.find_element_by_css_selector('img').get_attribute('src')
            )
            tour_list.append( obj )
    except Exception as e1:
        print( '오류', e1 )

print( tour_list, len(tour_list) )


1 페이지 이동
썸네임 http://tourimage.interpark.com/product/tour/00161/A30/280/A3015008_14_870.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3015008','')
상품명 [4국10일_대한항공]1급_융프라우_런던&로마 야간투어
코멘트 #베르사유궁전내부 #바티칸하이패스
가격 3,240,000 원~
여행 기간 : 8박10일
출발 가능 기간 : 2021.07.01~2021.12.30
평점 9.7


C:\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:689: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
C:\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:487: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
C:\anaconda3\lib\site-packages\selenium\webdriver\remote\webelement.py:505: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


15개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/A30/280/A3017982_1_870.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3017982','')
상품명 [2인이상 단독여행_이탈리아 8일] 로마+피렌체+베네치아 (항공제외)
코멘트 가이드 안내 + 자유 일정의 장점만 쏙쏙
가격 1,600,000 원~
여행 기간 : 6박7일
출발 가능 기간 : 2022.12.31~2022.12.31
평점 0.0
0개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/A30/280/A3017907_4_580.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3017907','')
상품명 [소그룹_서유럽 4국10일] 런던&로마 야간투어+융프라우
코멘트 ★소그룹_10명이상 출발 ! 최대 인원 15명 !★
가격 3,300,000 원~
여행 기간 : 8박10일
출발 가능 기간 : 2021.07.01~2021.12.30
평점 0.0
0개의 상품평
썸네임 http://tourimage.interpark.com/product/tour/00161/A30/280/A3016614_1_430.jpg
링크 searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3016614','')
상품명 [서유럽 10일]1급호텔_천공의 도시 치비타_융프라우_런던+로마야간
코멘트 치비타 디 바뇨레지오 내부
가격 3,100,000 원~
여행 기간 : 8박10일
출발 가능 기간 : 2021.07.01~2021.12.30
평점 0.0
0개의 상품평
썸네임 http://tourimage.int

In [9]:
# driver.close()

In [10]:
tour_list[5].link

"searchModule.OnClickDetail('http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3014578','')"

In [11]:
arr = tour_list[5].link.split(',')

In [12]:
link = arr[0].replace("searchModule.OnClickDetail('","")

In [13]:
detail_url = link[:-1]

In [14]:
detail_url

'http://tour.interpark.com/goods/detail/?BaseGoodsCd=A3014578'

In [15]:
driver.get(detail_url)

In [16]:
soup = bs(driver.page_source, 'html.parser')

In [17]:
data = soup.select('tip-cover')

In [20]:
content_final = data[0].contents

IndexError: list index out of range

In [10]:
# DB test  

In [11]:
if __name__=='__main__':
    db = DBHelper()
    print( db.db_selectKeyword() )
    print( db.db_insertCrawlingData('1','2','3','4','5') )
    db.db_free()


()
()
None
